In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install pickle5

In [ ]:
import pandas as pd
#import pickle5 as pickle
import pickle
import math

In [ ]:
with open('/content/drive/MyDrive/claim2fact/dictionary.pickle', 'rb') as f:
  data = pickle.load(f)

ValueError: ignored

In [ ]:
data = clean_data_new

In [ ]:
data[3]

{'cui': '14',
 'description': "In December 2015, several images were circulated on social media along with claims that they depicted presidential candidate Donald Trump’s sons, Eric and Donald Jr., on a big game hunting trip: The images were unaltered and accurately described, although they were not recent in December 2015. The photographs received broad media coverage at the time they were initially released, well before the start of Trump Sr.’s bid for the White House. On 13 March 2012, the web site Gothamist published an article titled “Photos: Donald Trump’s Sons Awesome At Killing Elephants And Other Wildlife.”  It said: Animal rights activists are revolted by a series of trophy photos that have emerged showing Eric and Donald Trump Jr. posing with a dead elephant, kudu, civet cat and waterbuck while on a big game safari in Zimbabwe last year. In one photo, Donald Jr. proudly holds a dead elephant tail in one hand and a knife in the other. In another, the brothers are seen standin

In [ ]:
data_restructured = {k:[] for k in data[0].keys()}

for item in data:
  if not isinstance(item['description'], str):
    continue
  for k in item.keys():
    data_restructured[k].append(str(item[k]))

In [ ]:
len(data_restructured['cui'])

3540

In [ ]:
!nvidia-smi

Sat Dec 11 23:17:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
import os
from datasets import Dataset
from tqdm.notebook import tqdm

In [ ]:
model_name='google/pegasus-cnn_dailymail'
device = 'cuda'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name, max_length=128).to(device)

In [ ]:
batch_size = 4
dataset = Dataset.from_dict(data_restructured)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
checkpoint_path = '/content/drive/MyDrive/claim2fact/dictionary_with_summaries.pkl'

In [ ]:
if os.path.exists(checkpoint_path):
  with open(checkpoint_path, 'rb') as f:
    final_dataset = pickle.load(f)
    completed_count = len(final_dataset['cui'])
else:
  final_dataset = {k:[] for k in data_restructured.keys()}
  final_dataset['summary'] = []
  completed_count = 0

In [ ]:
final_dataset

{'cui': [], 'description': [], 'summary': [], 'title': []}

In [ ]:
len(data_restructured['cui'])

3540

In [ ]:
for idx,batch in tqdm(enumerate(dataloader)):
  if completed_count != 0 and completed_count > idx * batch_size:
    continue
  text = batch['description']
  batch_tokens = tokenizer(text, truncation=True, padding='longest', return_tensors="pt").to(device)
  summaries = model.generate(**batch_tokens)
  tgt_text = tokenizer.batch_decode(summaries, skip_special_tokens=True)
  tgt_text = [clean_up(x) for x in tgt_text]
  for key in batch.keys():
    final_dataset[key].extend(batch[key])
  final_dataset['summary'].extend(tgt_text)

  if idx % 10 == 0 and idx != 0:
    with open(checkpoint_path, 'wb') as f:
      pickle.dump(final_dataset, f)

with open(checkpoint_path, 'wb') as f:
  pickle.dump(final_dataset, f)

0it [00:00, ?it/s]

TypeError: ignored

In [ ]:
data_with_summaries = []
for item in data:
  #item['cui'] = str(item['cui'])
  try:
    idx = final_dataset['cui'].index(item['cui'])
    summary = final_dataset['summary'][idx]
  except:
    summary = ""
  if not isinstance(item['description'], str):
    item['description'] = ""
  item['description'] = clean_up(item['description'])
  item['title'] = clean_up(item['title'])
  item['summary'] = summary
  data_with_summaries.append(item)


In [ ]:
data_with_summaries[:5]

[{'cui': '272',
  'description': 'As the US House of Representatives was readying a new special committee to investigate the terrorist attack in Benghazi Libya many Democrats were arguing that continuing to probe the Sept 11 2012 attack which killed four Americans including US Ambassador Chris Stevens amounted to a political witch hunt On May 5 2014 Rep John Garamendi DCalif told MSNBC host Ed Schultz that there has already been exhaustive testimony and investigation of the incident This thing is just going on and on to boredom actually Garamendi said The Armed Services Committee actually did a hearing and the result was theres nothing here Thats obviously a great tragedy but Ed during the George W Bush period there were 13 attacks on various embassies and consulates around the world Sixty people died In Karachi there was a death of one of our diplomats and those were not investigated during that period of time because it was a tragedy Readers asked us whether its true that under Bush 

In [ ]:
with open(checkpoint_path, "wb") as f:
  pickle.dump(data_with_summaries, f)

In [ ]:
with open(checkpoint_path, 'rb') as f:
    final_dataset = pickle.load(f)

In [ ]:
for item in final_dataset:
  item['title'] = clean_up(item['title'])

In [ ]:
final_dataset[:5]

[{'cui': '272',
  'description': 'As the US House of Representatives was readying a new special committee to investigate the terrorist attack in Benghazi Libya many Democrats were arguing that continuing to probe the Sept 11 2012 attack which killed four Americans including US Ambassador Chris Stevens amounted to a political witch hunt On May 5 2014 Rep John Garamendi DCalif told MSNBC host Ed Schultz that there has already been exhaustive testimony and investigation of the incident This thing is just going on and on to boredom actually Garamendi said The Armed Services Committee actually did a hearing and the result was theres nothing here Thats obviously a great tragedy but Ed during the George W Bush period there were 13 attacks on various embassies and consulates around the world Sixty people died In Karachi there was a death of one of our diplomats and those were not investigated during that period of time because it was a tragedy Readers asked us whether its true that under Bush 

In [ ]:
data_restructured['description'][:5]

['As the U.S. House of Representatives was readying a new special committee to investigate the terrorist attack in Benghazi, Libya, many Democrats were arguing that continuing to probe the Sept. 11, 2012, attack -- which killed four Americans, including U.S. Ambassador Chris Stevens -- amounted to a political witch hunt.\nOn May 5, 2014, Rep. John Garamendi, D-Calif., told MSNBC host Ed Schultz that there has already been exhaustive testimony and investigation of the incident.\n"This thing is just going on and on to boredom actually," Garamendi said. "The Armed Services Committee actually did a hearing and the result was there’s nothing here. That’s obviously a great tragedy, but Ed, during the George W. Bush period, there were 13 attacks on various embassies and consulates around the world. Sixty people died. In Karachi, there was a death of one of our diplomats, and those were not investigated during that period of time because it was a tragedy."\nReaders asked us whether it’s true t

In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import pickle
from urllib.error import HTTPError, URLError
import uuid

out_file = './article_body_titles.pkl'

val_file = 'val.json'
train_file = 'train.json'
test_file = 'test.json'
site_main_tag = {'politifact': 'm-textblock', 'snopes': "single-body card card-body rich-text"} # A bit hacky but works
val_df = pd.read_json(val_file)
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)

url_list = train_df['url'].to_list()
url_list.extend(val_df['url'].to_list())
url_list.extend(test_df['url'].to_list())
url_list = list(set(url_list))

failed_list = []
CLEANR = re.compile(r"(\n|\xa0)+")
fetched_data = []

for url in tqdm(url_list):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        webpage = urlopen(req).read()
        content = webpage.decode('utf-8')
        soup = BeautifulSoup(content, 'html.parser')
        for site in site_main_tag.keys():
            if site in url:
                body = soup.find(class_=site_main_tag[site]).get_text()
        body = re.sub(CLEANR, ' ', body).strip()
        title = soup.find('title').get_text()
        fetched_data.append({'body': body, 'title': title})
    except (URLError, HTTPError) as e:
        failed_list.append(url)

print(f"Failed to fetch {len(failed_list)} URLs")

with open(out_file, 'wb') as f:
    pickle.dump(fetched_data, f)

  0%|          | 20/4319 [00:21<1:15:33,  1.05s/it]


KeyboardInterrupt: ignored

In [ ]:
with open('./dictionary.pickle', 'rb') as f:
  clean_data = pickle.load(f)

In [ ]:
clean_data[0]

{'cui': '272',
 'description': 'As the US House of Representatives was readying a new special committee to investigate the terrorist attack in Benghazi Libya many Democrats were arguing that continuing to probe the Sept 11 2012 attack which killed four Americans including US Ambassador Chris Stevens amounted to a political witch huntOn May 5 2014 Rep John Garamendi DCalif told MSNBC host Ed Schultz that there has already been exhaustive testimony and investigation of the incidentThis thing is just going on and on to boredom actually Garamendi said The Armed Services Committee actually did a hearing and the result was theres nothing here Thats obviously a great tragedy but Ed during the George W Bush period there were 13 attacks on various embassies and consulates around the world Sixty people died In Karachi there was a death of one of our diplomats and those were not investigated during that period of time because it was a tragedyReaders asked us whether its true that under Bush there

In [ ]:
with open('./article_body_titles.pkl', 'rb') as f:
  articles = pickle.load(f)

In [ ]:
import json
with open('./url_cui_mapping.json') as f:
  url_map = json.load(f)

In [ ]:
import re
def clean_up(s: str):
    if type(s) is float and math.isnan(s):
        return ""
    else:
        s = re.sub("\n"," ",s)
        s = ''.join(e for e in s if e.isalnum() or e is ' ')
        s = re.sub("\s+"," ",s)
        return s.strip()

In [ ]:
articles[0]

{'body': 'In February 2017, a December 2015 commentary that originally aired on a Georgia television newscast resurfaced online, published under misleading headlines implying that President Donald Trump had just banned “Sharia law” in the United States.  Multiple web sites published (sometimes without attribution) transcripts of a segment by TV anchor Ben Swann of Atlanta television station WGCL discussing then-candidate Donald Trump’s statement calling for a “total and complete shutdown” of Muslim immigration into the U.S. Trump said the restriction was necessary, because (in his words) “large portions” of Muslim communities harbor “great hatred towards Americans”: Until recently we are able to determine and understand this problem and the dangerous threat it poses, our country cannot be the victims of horrendous attacks by people that believe only in Jihad, and have no sense of reason or respect for human life. If I win the election for President, we are going to Make America Great A

In [ ]:
all_data = pd.read_csv('all_data_with_article_title_body.csv')

In [ ]:
clean_data_new = []
for item in clean_data:
  cui = item['cui']
  url = cui_map[cui]
  new_item = {}
  new_item['cui'] = cui
  content = list(filter(lambda article: article['url'] == url, articles))[0]
  new_item['title'] = clean_up(content['title'])
  new_item['description'] = clean_up(content['body'])
  clean_data_new.append(new_item)

In [ ]:
clean_data_new = []
for item in clean_data:
  cui = item['cui']
  url = cui_map[cui]
  new_item = {}
  new_item['cui'] = cui
  content = list(filter(lambda article: article['url'] == url, articles))[0]
  new_item['title'] = content['title']
  new_item['description'] = content['body']
  clean_data_new.append(new_item)

In [ ]:
clean_data_new[3]

{'cui': '14',
 'description': "In December 2015, several images were circulated on social media along with claims that they depicted presidential candidate Donald Trump’s sons, Eric and Donald Jr., on a big game hunting trip: The images were unaltered and accurately described, although they were not recent in December 2015. The photographs received broad media coverage at the time they were initially released, well before the start of Trump Sr.’s bid for the White House. On 13 March 2012, the web site Gothamist published an article titled “Photos: Donald Trump’s Sons Awesome At Killing Elephants And Other Wildlife.”  It said: Animal rights activists are revolted by a series of trophy photos that have emerged showing Eric and Donald Trump Jr. posing with a dead elephant, kudu, civet cat and waterbuck while on a big game safari in Zimbabwe last year. In one photo, Donald Jr. proudly holds a dead elephant tail in one hand and a knife in the other. In another, the brothers are seen standin

In [ ]:
with open('dictionary_full_no_summaries.pkl', 'wb') as f:
  pickle.dump(clean_data_new, f)

In [ ]:
cui_map = {}
for a in url_map.keys():
  cui_map[url_map[a]] = a

In [ ]:
cui_map

{'272': 'http://www.politifact.com/truth-o-meter/statements/2014/may/12/john-garamendi/prior-benghazi-were-there-13-attacks-embassies-and/',
 '273': 'http://www.politifact.com/truth-o-meter/statements/2012/sep/07/barack-obama/barack-obama-said-hes-cut-taxes-middle-class-famil/',
 '274': 'http://www.politifact.com/punditfact/statements/2014/aug/24/juan-williams/juan-williams-no-1-cause-death-african-americans-1/',
 '14': 'https://www.snopes.com/fact-check/donald-trumps-son-game-hunting-photos/',
 '275': 'http://www.politifact.com/truth-o-meter/statements/2016/oct/19/hillary-clinton/hillary-clinton-blames-russia-putin-wikileaks-rele/',
 '72': 'https://www.snopes.com/fact-check/promotional-booklet/',
 '2': 'https://www.snopes.com/business/consumer/bastard.asp',
 '276': 'http://www.snopes.com/obama-more-flexibility-russia/',
 '277': 'https://www.snopes.com/fact-check/did-roseanne-pose-as-adolf-hitler-photo-shoot/',
 '278': 'http://www.politifact.com/punditfact/statements/2016/aug/28/david-

In [ ]:
with open('dictionary_with_summaries_all.pkl', 'rb') as f:
  summaries = pickle.load(f)

In [ ]:
summaries[0]

{'cui': '10930383388364076528',
 'description': 'As the U.S. House of Representatives was readying a new special committee to investigate the terrorist attack in Benghazi, Libya, many Democrats were arguing that continuing to probe the Sept. 11, 2012, attack -- which killed four Americans, including U.S. Ambassador Chris Stevens -- amounted to a political witch hunt.\nOn May 5, 2014, Rep. John Garamendi, D-Calif., told MSNBC host Ed Schultz that there has already been exhaustive testimony and investigation of the incident.\n"This thing is just going on and on to boredom actually," Garamendi said. "The Armed Services Committee actually did a hearing and the result was there’s nothing here. That’s obviously a great tragedy, but Ed, during the George W. Bush period, there were 13 attacks on various embassies and consulates around the world. Sixty people died. In Karachi, there was a death of one of our diplomats, and those were not investigated during that period of time because it was a 

In [ ]:
with open('dictionary_full_no_summaries.pkl', 'rb') as f:
  final = pickle.load(f)

In [ ]:
final[3]

{'cui': '14',
 'description': 'In December 2015 several images were circulated on social media along with claims that they depicted presidential candidate Donald Trumps sons Eric and Donald Jr on a big game hunting trip The images were unaltered and accurately described although they were not recent in December 2015 The photographs received broad media coverage at the time they were initially released well before the start of Trump Srs bid for the White House On 13 March 2012 the web site Gothamist published an article titled Photos Donald Trumps Sons Awesome At Killing Elephants And Other Wildlife It said Animal rights activists are revolted by a series of trophy photos that have emerged showing Eric and Donald Trump Jr posing with a dead elephant kudu civet cat and waterbuck while on a big game safari in Zimbabwe last year In one photo Donald Jr proudly holds a dead elephant tail in one hand and a knife in the other In another the brothers are seen standing beside a 128 crocodile han